In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-and-test-dataset/train.csv
/kaggle/input/train-and-test-dataset/test.csv
/kaggle/input/barbel-curl-preprocessed-dataset/pose_data_preprocessed.csv


In [7]:
!pip install tensorflow

In [8]:
import os;
os.listdir('/kaggle/input/')

['train-and-test-dataset', 'barbel-curl-preprocessed-dataset']

In [9]:
df = pd.read_csv("/kaggle/input/train-and-test-dataset/train.csv")

In [10]:
df.head(10)

,NOSE_x,NOSE_y,NOSE_z,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,RIGHT_SHOULDER_x,RIGHT_SHOULDER_y,RIGHT_SHOULDER_z,RIGHT_ELBOW_x,...,LEFT_WRIST_x,LEFT_WRIST_y,LEFT_WRIST_z,LEFT_HIP_x,LEFT_HIP_y,LEFT_HIP_z,RIGHT_HIP_x,RIGHT_HIP_y,RIGHT_HIP_z,label
0,0.471012,0.153530,0.500676,0.521585,0.143661,0.582902,0.461298,0.174968,0.462398,0.556141,...,0.588857,0.277502,0.379209,0.640369,0.312708,0.573282,0.619768,0.430786,0.425827,0
1,0.467903,0.164373,0.534046,0.517059,0.148057,0.606298,0.458718,0.173553,0.489304,0.551948,...,0.577198,0.291355,0.419041,0.637866,0.317643,0.571737,0.619003,0.442024,0.427426,0
2,0.145693,0.291957,0.486431,0.403305,0.233623,0.280822,0.328959,0.308854,0.567276,0.518348,...,0.569074,0.541731,0.315905,0.661139,0.381344,0.198381,0.617833,0.530057,0.799713,1
3,0.518484,0.121850,0.600098,0.589210,0.225711,0.452384,0.589784,0.233873,0.608781,0.678253,...,0.522680,0.647569,0.480614,0.710686,0.469862,0.341718,0.697406,0.627967,0.657904,0
4,0.462099,0.269903,0.525970,0.504655,0.236624,0.594655,0.457422,0.268088,0.481632,0.547376,...,0.561069,0.374897,0.407708,0.628349,0.369815,0.580552,0.611286,0.510722,0.418547,0
5,0.466853,0.217060,0.597817,0.455731,0.212778,0.716696,0.447873,0.256840,0.413323,0.567682,...,0.539500,0.373720,0.466399,0.620947,0.417518,0.761371,0.611909,0.578809,0.238505,0
6,0.421290,0.264076,0.573826,0.578699,0.223772,0.541517,0.426047,0.273328,0.680219,0.499004,...,0.440796,0.337602,0.345404,0.650514,0.372055,0.293668,0.598742,0.520218,0.704541,1
7,0.154324,0.305016,0.469458,0.412502,0.238016,0.256185,0.338124,0.310242,0.559372,0.526422,...,0.567710,0.541554,0.321745,0.668296,0.382601,0.197353,0.621346,0.533142,0.800785,1
8,0.491661,0.221582,0.607879,0.426833,0.228881,0.740459,0.456774,0.269948,0.344908,0.553820,...,0.529966,0.544817,0.602154,0.596217,0.459087,0.865903,0.607195,0.639638,0.134177,0
9,0.423937,0.192926,0.471831,0.530395,0.287871,0.591094,0.359082,0.330039,0.460979,0.473077,...,0.597147,0.806057,0.463833,0.636041,0.634035,0.585350,0.561427,0.867945,0.415771,0


In [11]:
df.shape

(115000, 28)

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt

In [13]:
# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [14]:
X = df.iloc[:, :-1]  # Features (all columns except last)
y = df.iloc[:, -1]   # Target (last column)

# Split dataset into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print shapes
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (80500, 27) (80500,)
Testing set shape: (34500, 27) (34500,)


BUILD MODEL

3.1. Set up

In [15]:
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [16]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

3.2. Model with 3 layers

In [17]:
print("X_train shape:", X_train.shape)  # Should be (num_samples, num_features)

X_train shape: (80500, 27)


In [18]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [19]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(27, input_dim=27, activation="relu"))  # Change 36 to 27

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation="softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])
    
    return model


In [20]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 01m 03s]
val_accuracy: 0.99924635887146

Best val_accuracy So Far: 0.9995942115783691
Total elapsed time: 00h 12m 48s


In [21]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 128 units, func:  <function relu at 0x7904b1f27910>
Layer-3: 2 units, func:  <function softmax at 0x79045260af80>

Other params:
learning_rate: 0.001
Epoch 1/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9777 - loss: 0.0586 - val_accuracy: 0.9989 - val_loss: 0.0035
Epoch 2/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9983 - loss: 0.0066 - val_accuracy: 0.9992 - val_loss: 0.0023
Epoch 3/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9986 - loss: 0.0050 - val_accuracy: 0.9994 - val_loss: 0.0021
Epoch 4/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9989 - loss: 0.0042 - val_accuracy: 0.9994 - val_loss: 0.0014
Epoch 5/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9989 - loss: 0.0039 - val_accuracy: 0.9939 - val_loss: 0.0259
Epoch 6/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9990 - loss: 0.0036 - 

In [22]:
final_models["3_layers"] = model_3l

3.3. Model with 5 layers

In [23]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(27, input_dim = 27, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [24]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 39s]
val_accuracy: 0.9992173910140991

Best val_accuracy So Far: 0.99962317943573
Total elapsed time: 00h 16m 14s


In [25]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 416 units, func:  <function relu at 0x7904b1f27910>
Layer-3: 416 units, func:  <function relu at 0x7904b1f27910>
Layer-4: 288 units, func:  <function relu at 0x7904b1f27910>
Layer-5: 2 units, func:  <function softmax at 0x79045260af80>

Other params:
learning_rate: 0.0001
Epoch 1/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.9732 - loss: 0.0709 - val_accuracy: 0.9990 - val_loss: 0.0043
Epoch 2/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9983 - loss: 0.0078 - val_accuracy: 0.9982 - val_loss: 0.0066
Epoch 3/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9986 - loss: 0.0050 - val_accuracy: 0.9992 - val_loss: 0.0021


In [26]:
final_models["5_layers"] = model_5l

3.4. Model with 7 layers include Dropout

In [27]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(27, input_dim = 27, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [28]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 43s]
accuracy: 0.9976397752761841

Best accuracy So Far: 0.9989565014839172
Total elapsed time: 00h 17m 42s


In [29]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 256 units, func:  <function relu at 0x7904b1f27910>
Layer-3: 0 units, func: None
Layer-4: 128 units, func:  <function relu at 0x7904b1f27910>
Layer-5: 0 units, func: None
Layer-6: 512 units, func:  <function relu at 0x7904b1f27910>
Layer-7: 2 units, func:  <function softmax at 0x79045260af80>

Other params:
dropout_1: 0.5
dropout_2: 0.1
learning_rate: 0.0001
Epoch 1/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - accuracy: 0.9282 - loss: 0.1515 - val_accuracy: 0.9975 - val_loss: 0.0094
Epoch 2/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9978 - loss: 0.0092 - val_accuracy: 0.9990 - val_loss: 0.0031
Epoch 3/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9983 - loss: 0.0067 - val_accuracy: 0.9993 - val_loss: 0.0020


In [30]:
final_models["7_layers_with_dropout"] = model_7lD

3.5. Model with 7 layers

In [31]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(27, input_dim = 27, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [32]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 42s]
val_accuracy: 0.9995072484016418

Best val_accuracy So Far: 0.9995072484016418
Total elapsed time: 00h 23m 14s


In [33]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 160 units, func:  <function tanh at 0x79045260b370>
Layer-3: 160 units, func:  <function tanh at 0x79045260b370>
Layer-4: 96 units, func:  <function tanh at 0x79045260b370>
Layer-5: 32 units, func:  <function tanh at 0x79045260b370>
Layer-6: 320 units, func:  <function tanh at 0x79045260b370>
Layer-7: 2 units, func:  <function softmax at 0x79045260af80>

Other params:
learning_rate: 0.001
Epoch 1/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9877 - loss: 0.0381 - val_accuracy: 0.9991 - val_loss: 0.0056
Epoch 2/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9975 - loss: 0.0132 - val_accuracy: 0.9987 - val_loss: 0.0056
Epoch 3/100
8050/8050 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9980 - loss: 0.0104 - val_accuracy: 0.9991 - val_loss: 0.0046


In [34]:
final_models["7_layers"] = model_7l

3.6. Describe final models

In [35]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 128 units, func:  <function relu at 0x7904b1f27910>
Layer-3: 2 units, func:  <function softmax at 0x79045260af80>

5_layers: Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 416 units, func:  <function relu at 0x7904b1f27910>
Layer-3: 416 units, func:  <function relu at 0x7904b1f27910>
Layer-4: 288 units, func:  <function relu at 0x7904b1f27910>
Layer-5: 2 units, func:  <function softmax at 0x79045260af80>

7_layers_with_dropout: Describe models architecture
Layer-1: 27 units, func:  <function relu at 0x7904b1f27910>
Layer-2: 256 units, func:  <function relu at 0x7904b1f27910>
Layer-3: 0 units, func: None
Layer-4: 128 units, func:  <function relu at 0x7904b1f27910>
Layer-5: 0 units, func: None
Layer-6: 512 units, func:  <function relu at 0x7904b1f27910>
Layer-7: 2 units, func:  <function softmax at 0x79045260af80>

7_layers: Descri

4. Model Evaluation
4.1. Train set evaluation

In [36]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

# Function to round metric results
def round_up_metric_results(scores, decimals=4):
    return [round(score, decimals) for score in scores]
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(X_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.9997, 0.9997]","[0.9998, 0.9995]","[0.9997, 0.9996]","[[20536, 4], [7, 13953]]"
1,7_layers_with_dropout,"[0.9991, 0.9997]","[0.9998, 0.9986]","[0.9994, 0.9992]","[[20536, 4], [19, 13941]]"
2,5_layers,"[0.9987, 0.9999]","[1.0, 0.9981]","[0.9993, 0.999]","[[20539, 1], [27, 13933]]"
3,7_layers,"[0.9986, 0.9999]","[0.9999, 0.998]","[0.9993, 0.9989]","[[20538, 2], [28, 13932]]"


4.2. Test set evaluation

In [38]:
# load dataset
test_df = pd.read_csv("/kaggle/input/train-and-test-dataset/test.csv")

In [40]:
test_x = test_df.drop("label", axis = 1)

In [41]:
test_y = test_df["label"]

In [46]:
# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.9996, 0.9991]","[0.9994, 0.9995]","[0.9995, 0.9993]","[[17121, 10], [6, 11614]]"
1,7_layers_with_dropout,"[0.9993, 0.999]","[0.9993, 0.999]","[0.9993, 0.999]","[[17119, 12], [12, 11608]]"
2,5_layers,"[0.999, 0.9997]","[0.9998, 0.9985]","[0.9994, 0.9991]","[[17128, 3], [17, 11603]]"
3,7_layers,"[0.9988, 0.9994]","[0.9996, 0.9982]","[0.9992, 0.9988]","[[17124, 7], [21, 11599]]"


5. Dumped Model

In [48]:
import os
import pickle

# Ensure the directory exists
os.makedirs("./model", exist_ok=True)  # Creates the "model" directory if it doesn't exist

# Save the model
with open("./model/bicep_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers"], f)

print("Model saved successfully!")


Model saved successfully!


In [52]:
# Dump final results
with open("./model/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)

In [54]:
# Load all deep learning models
with open("/kaggle/working/model/all_models.pkl", "rb") as f:
    dp_models = pickle.load(f)

In [58]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc

In [59]:
test_set_results = []

for name, model in dp_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    p_score = precision_score(y_test_class, y_pred_class, average="macro")
    a_score = accuracy_score(y_test_class, y_pred_class)
    r_score = recall_score(y_test_class, y_pred_class, average="macro")
    f1_score_result = f1_score(y_test_class, y_pred_class, average="macro")
    
    test_set_results.append(( name,  (p_score), r_score, (a_score), (f1_score_result), cm ))

dp_eval = pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "Accuracy Score", "F1 Score", "Confusion Matrix"])

dp_eval

,Model,Precision Score,Recall Score,Accuracy Score,F1 Score,Confusion Matrix
0,3_layers,0.999395,0.999450,0.999443,0.999422,"[[17121, 10], [6, 11614]]"
1,5_layers,0.999375,0.999181,0.999304,0.999278,"[[17128, 3], [17, 11603]]"
2,7_layers_with_dropout,0.999133,0.999133,0.999165,0.999133,"[[17119, 12], [12, 11608]]"
3,7_layers,0.999086,0.998892,0.999026,0.998989,"[[17124, 7], [21, 11599]]"
